In [1]:
pip install pennylane

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.1/1.8 MB 812.7 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.8 MB 919.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.8 MB 726.2 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.8 MB 654.9 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.8 MB 607.9 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/1.8 MB 583.1 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/1.8 MB 583.1 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 541.9 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.8 MB 550.0 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.8 MB 537.7 kB/s eta 0:00:03
   ------ --------------------------------- 0.3/1.8 MB 532.5 kB/s eta 0:00:

In [3]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Set random seed for reproducibility
np.random.seed(42)

# 1. Discretize the interval [0, 2π]
num_points = 50
x_values = np.linspace(0, 2 * np.pi, num_points)
y_values = np.sin(x_values)  # True sine values

In [7]:
# 2. Quantum Feature Map - Encoding x into quantum states
def feature_map(x):
    qml.Hadamard(wires=0)
    qml.RX(x, wires=0)
    qml.RZ(x, wires=0)

# 3. Variational Quantum Circuit (VQC)
def variational_circuit(params):
    qml.RY(params[0], wires=0)
    qml.RZ(params[1], wires=0)

# 4. QNode - combining feature map and VQC
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def quantum_model(x, params):
    feature_map(x)
    variational_circuit(params)
    return qml.expval(qml.PauliZ(0))

# 5. Cost Function
def cost(params, x, y):
    predictions = np.array([quantum_model(x_i, params) for x_i in x])
    return np.mean((predictions - y) ** 2)

In [9]:
# 6. Optimization
params = np.random.randn(2)  # Random initial parameters
opt = qml.GradientDescentOptimizer(stepsize=0.1)
num_iterations = 100
cost_values = []

for it in range(num_iterations):
    params = opt.step(lambda v: cost(v, x_values, y_values), params)
    cost_value = cost(params, x_values, y_values)
    cost_values.append(cost_value)
    if it % 10 == 0:
        print(f"Iteration {it}: Cost = {cost_value:.4f}")

Iteration 0: Cost = 0.6066
Iteration 10: Cost = 0.5074
Iteration 20: Cost = 0.4921
Iteration 30: Cost = 0.4902
Iteration 40: Cost = 0.4900
Iteration 50: Cost = 0.4900
Iteration 60: Cost = 0.4900
Iteration 70: Cost = 0.4900
Iteration 80: Cost = 0.4900
Iteration 90: Cost = 0.4900


In [ ]:
# 7. Predictions and Plotting
predicted_values = np.array([quantum_model(x, params) for x in x_values])

# Plotting the results
plt.plot(x_values, y_values, label="True Sine", color="blue")
plt.plot(x_values, predicted_values, label="Quantum Model", color="red", linestyle="dashed")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.title("Sine Function Approximation with Quantum Model")
plt.legend()
plt.show()